In [ ]:
from urllib.request import Request, urlopen
from requests_html import HTML
import json
import threading

In [ ]:
def get_complete_file(URL):
    request = Request(URL, headers={'User-Agent': 'chrome/5.0'})
    web_byte = urlopen(request).read()
    webpage = web_byte.decode('utf-8')
    html_text = HTML(html=webpage)
    final_object = {}
    final_object["title"] = html_text.find("h1[class='content__title']")[0].text
    final_object["sub_title"] = html_text.find("div[class='content__category']")[0].text.split("|")[0].strip()
    final_object["description"] = html_text.find("div[class='description-container']")[0].text
    final_object["source_url"] = URL
    final_object["image_url"] = ""

    all_properties = html_text.find("div[class='single-recipe-page-info-item clearfix']")
    for i in all_properties:
        heading = i.find("div[class='single-recipe-page-info-item__title']")[0].text.lower().replace(" ", "_")
        text = i.find("div[class='single-recipe-page-info-item__text']")[0].text.lower()
        final_object[heading] = text

    final_object["ingredients"] = []
    lis = html_text.find("ul[class='single-recipe-page-ingredients__list']")[0].find("li")
    for index, k in enumerate(lis):
        obj = {}
        text = k.text
        array = text.split(" ")
        qty = (array[0])
        amount = 0
        unit = ""
        index = 0
        try:
            amount = float(array[1])
            unit = array[2]
            index = 3
        except ValueError:
            amount = ""
            unit = ""
            index = 1
        except IndexError:
            amount = ""
            unit = ""
            index = 0

        obj["name"] = " ".join(array[index::])
        obj["amount"] = amount
        obj["unit"] = unit
        obj["qty"] = qty
        obj["sort_order"] = index

        final_object["ingredients"].append(obj)


    final_object["steps"] = []

    steps_list = html_text.find("ul[class='single-recipe-page-steps-container']")[0].find("li")
    for index, j in enumerate(steps_list):
        title = j.find("div[class='single-recipe-page-step__title pull-left']")[0].text.split(".")[1].strip()
        description = j.find("div[class='single-recipe-page-step__text']")[0].text

        obj = {}
        obj["title"] = title
        obj["text"] = description
        obj["sort_order"] = index

        final_object["steps"].append(obj)
        
        t = final_object["title"]
        not_allowed_character = ["\\", "/", ":", "*", "?", "\"", "<", ">", "|", "&"]
        for single_n in not_allowed_character:
            if single_n in t:
                t.replace(single_n, " ")

    with open(f'./output-folder/{t}.json', "w", encoding="utf-8") as file:
        json.dump(final_object, file, ensure_ascii=False)

In [ ]:
def get_all_links():
    complete_list = []
    MAIN_URL = "https://soyummy.com/"
    all_categories = ["breakfast", "dinner", "desserts", "beverages"]
    for i in all_categories:
        URL = MAIN_URL + i
        request = Request(URL, headers={'User-Agent': 'chrome/5.0'})
        web_byte = urlopen(request).read()
        webpage = web_byte.decode('utf-8')
        html_text = HTML(html=webpage)
        all_a = html_text.find("a[class='post-preview-box__link']")
        for single_link in all_a:
            complete_list.append(single_link.attrs["href"])
    return complete_list

def write_json():
    complete_list = get_all_links()
    with open("./input-folder/all-links-file.json", "w", encoding="utf-8") as json_file:
        json.dump(complete_list, json_file, ensure_ascii=False)

In [ ]:
def get_json_file_list():
    with open("./input-folder/all-links-file.json", encoding="utf-8") as file:
        return json.load(file)
    return None

def get_files_from_links(*json_list):
    for single_link in json_list:
        get_complete_file(single_link)